In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install --upgrade /content/drive/MyDrive/multilingual_rouge_scoring


In [ ]:
!pip install git+https://github.com/otuncelli/turkish-stemmer-python
!pip install git+https://github.com/abhik1505040/bengali-stemmer
!pip install absl-py nltk numpy six>=1.14 pythainlp pyonmttok jieba fugashi[unidic]

In [ ]:
!pip install datasets

!pip install rouge_score

In [ ]:
!pip install bert-score

In [ ]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
from rouge_score import rouge_scorer
import statistics
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, AlbertTokenizer
import re
import nltk
!pip install datasets
from bert_score import score as bert_score



In [ ]:
import nltk
from nltk.translate import meteor_score
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Download necessary NLTK resources
nltk.download('wordnet')
nltk.download('punkt')

In [ ]:
# Load the trained model and tokenizer from the Hugging Face model hub
model_name = "mdosama39/mbart-large-50-headline_WithIp_Aspect_Only"  # Replace with your model's name

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
# Load the evaluation dataset
dataset = load_dataset("osama39/BNHG", split="test")  # Replace with your dataset name and split if different

In [ ]:
df = dataset.to_pandas()

In [ ]:
!pip install git+https://github.com/Hasan-Mesbaul-Ali-Taher/normalizer
from normalizer import normalize
import re

In [ ]:
def preprocess_text(text):

    # Normalize the text using BUET normalizer
    text = normalize(text, unicode_norm="NFKC", punct_replacement= None, url_replacement=" ", emoji_replacement=" ", apply_unicode_norm_last=True)

    # # Define the Bengali Unicode range (Bengali script starts at U+0980)
    # bengali_range = "\u0980-\u09FF"

    # # Remove all characters that are not Bengali words or punctuations
    # processed_text = re.sub(f"[^{bengali_range}!\"#$%&'()*+,-./:;<=>?@[\\]^_`{{|}}~।\s]", "", text)

    # Replace consecutive punctuations with whitespace in between with only the first punctuation
    # processed_text = re.sub(r"([!\"#$%&'()*+,-./:;<=>?@[\\]^_`{{|}}~।])\\1+", r"\1", processed_text)
    processed_text =  re.sub(r'([.,?!|।^_`#@{{|}}~।#$%\&()*+,-./:;<=>?@])\1+', r'\1', text)
    # Split the text into words and remove empty strings caused by consecutive spaces
    words = processed_text.split()
    # Join the words with a single space to form the cleaned text
    text = ' '.join(words)

    result = []
    prev_char = ''

    for char in text:
        if char.strip() == prev_char.strip() and char.strip() in "!।|@#$%^&*()_+-=[]{}|;':\",./<>?.,?!|।^_`#@{{|}}~।#$%\&()*+,-./:;<=>?@":
            continue
        result.append(char)
        prev_char = char

    modified_text = ''.join(result)
    # Define a regular expression to find consecutive punctuations separated by space
    pattern = r'(\s*[\]\[\।\!\@\#\$\%\^\&\*\(\)\_\+\-\=\[\]\{\}\|\;\'\:\"\,\.\/\<\>\?\.\,\?\!\|\।\^\_\`\#\@\{\{\|\}\}\~\।\#\$\%\\\&\(\)\*\+\,\-\.\/\:\;\<\=\>\?\@]+)+\s*'

    # Use re.sub() to replace consecutive punctuations with a single punctuation
    processed_text = re.sub(pattern, r'\1', modified_text)
    # Remove leading and lagging spaces
    processed_text = processed_text.strip()
    return processed_text

In [ ]:
df['Article'] = df['Article'].apply(lambda text: preprocess_text(text))


In [ ]:
from datasets import Dataset

# Convert the Pandas DataFrame back to a Hugging Face dataset
dataset = Dataset.from_pandas(df)

In [ ]:
# Define tokenizer and BLEU score functions
_WORD_SPLIT = re.compile("([.,!?\"':;)(])")

def tokenizer_basic(sentence):
    """Very basic tokenizer: split the sentence into a list of tokens."""
    words = []
    for space_separated_fragment in sentence.strip().split():
        words.extend(re.split(_WORD_SPLIT, space_separated_fragment))
    return [w for w in words if w]

def getBLEUscore(true_summary, predicted_summary):
    token_true_summary = []
    for sentence in true_summary:
        token_true_summary.append(tokenizer_basic(sentence))

    token_predicted_summary = []
    for sentence in predicted_summary:
        token_predicted_summary.append(tokenizer_basic(sentence))

    BLEUscore = []
    weights = [1, 0, 0, 0]  # Weights for unigrams only
    total_predicted_summary = len(predicted_summary)

    for id in range(total_predicted_summary):
        BLEUscore.append(nltk.translate.bleu_score.sentence_bleu(
            [token_true_summary[id]], token_predicted_summary[id], weights=weights))

    avgBLEUscore = sum(BLEUscore) / total_predicted_summary
    return BLEUscore, avgBLEUscore

In [ ]:
# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True,lang="bengali")

# Lists to store ROUGE scores
rouge1_precision = []
rouge1_recall = []
rouge1_fmeasure = []

rouge2_precision = []
rouge2_recall = []
rouge2_fmeasure = []

rougeL_precision = []
rougeL_recall = []
rougeL_fmeasure = []

In [ ]:
# List to store METEOR scores
meteor_scores = []

In [ ]:
bert_precisions = []
bert_recalls = []
bert_f1s = []

In [ ]:
!pip install nltk
import nltk
from nltk.translate.meteor_score import single_meteor_score


In [ ]:
# Initialize scorer for ROUGE
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True,lang="bengali")

# Initialize lists to store scores
meteor_scores = []
bert_precisions = []
bert_recalls = []
bert_f1s = []
rouge1_precision = []
rouge1_recall = []
rouge1_fmeasure = []
rouge2_precision = []
rouge2_recall = []
rouge2_fmeasure = []
rougeL_precision = []
rougeL_recall = []
rougeL_fmeasure = []
true_summaries = []
predicted_summaries = []
# # Evaluate the model on the dataset for baseline
# for example in dataset:
#     inputs = tokenizer(example["Article"], return_tensors="pt", max_length=1024, truncation=True)

#     inputs.pop('token_type_ids', None)  # Remove 'token_type_ids' if present
#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(example["Headlines"], return_tensors="pt", max_length=15, truncation=True)


# Evaluate the model on the dataset for MultiGen
for example in dataset:
    # Prepare inputs
    inputs_text = example["Category"] + " [SEP] " + example["Aspect"] + " [SEP] " + example["Sentiment"] + " [SEP] " + example["Article"]
    inputs = tokenizer(inputs_text, return_tensors="pt", max_length=1024, truncation=True)
    inputs.pop('token_type_ids', None)  # Remove 'token_type_ids' if present

    # Generate summary
    outputs = model.generate(**inputs)
    generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    true_summary = example["Headlines"]

    # Store true and predicted summaries
    true_summaries.append(true_summary)
    predicted_summaries.append(generated_summary)

    # # Compute METEOR score
    # reference_tokens = nltk.word_tokenize(true_summary)
    # generated_summary_tokens = nltk.word_tokenize(generated_summary)
    # meteor = single_meteor_score(" ".join(reference_tokens), " ".join(generated_summary_tokens))
    # meteor_scores.append(meteor)

    # Compute METEOR score
    reference_tokens = nltk.word_tokenize(true_summary)
    generated_summary_tokens = nltk.word_tokenize(generated_summary)
    meteor = single_meteor_score(reference_tokens, generated_summary_tokens) # Call the imported function
    meteor_scores.append(meteor)

    # Compute BERTScore
    P, R, F1 = bert_score([generated_summary], [true_summary], lang='bn')
    bert_precisions.append(P.mean().item())
    bert_recalls.append(R.mean().item())
    bert_f1s.append(F1.mean().item())

    # Compute ROUGE scores
    scores = scorer.score(true_summary, generated_summary)
    rouge1_precision.append(scores['rouge1'].precision)
    rouge1_recall.append(scores['rouge1'].recall)
    rouge1_fmeasure.append(scores['rouge1'].fmeasure)
    rouge2_precision.append(scores['rouge2'].precision)
    rouge2_recall.append(scores['rouge2'].recall)
    rouge2_fmeasure.append(scores['rouge2'].fmeasure)
    rougeL_precision.append(scores['rougeL'].precision)
    rougeL_recall.append(scores['rougeL'].recall)
    rougeL_fmeasure.append(scores['rougeL'].fmeasure)

# Compute mean scores
mean_meteor_score = mean(meteor_scores)
mean_bert_precision = mean(bert_precisions)
mean_bert_recall = mean(bert_recalls)
mean_bert_f1 = mean(bert_f1s)
mean_rouge1_precision = mean(rouge1_precision)
mean_rouge1_recall = mean(rouge1_recall)
mean_rouge1_fmeasure = mean(rouge1_fmeasure)
mean_rouge2_precision = mean(rouge2_precision)
mean_rouge2_recall = mean(rouge2_recall)
mean_rouge2_fmeasure = mean(rouge2_fmeasure)
mean_rougeL_precision = mean(rougeL_precision)
mean_rougeL_recall = mean(rougeL_recall)
mean_rougeL_fmeasure = mean(rougeL_fmeasure)

# Calculate BLEU scores
BLEUscore, avgBLEUscore = getBLEUscore(true_summaries, predicted_summaries)

# Print mean scores
print(f"ROUGE-1 Precision: {mean_rouge1_precision:.4f}")
print(f"ROUGE-1 Recall: {mean_rouge1_recall:.4f}")
print(f"ROUGE-1 F-measure: {mean_rouge1_fmeasure:.4f}")
print()
print(f"ROUGE-2 Precision: {mean_rouge2_precision:.4f}")
print(f"ROUGE-2 Recall: {mean_rouge2_recall:.4f}")
print(f"ROUGE-2 F-measure: {mean_rouge2_fmeasure:.4f}")
print()
print(f"ROUGE-L Precision: {mean_rougeL_precision:.4f}")
print(f"ROUGE-L Recall: {mean_rougeL_recall:.4f}")
print(f"ROUGE-L F-measure: {mean_rougeL_fmeasure:.4f}")
print()
print(f"Average BLEU Score: {avgBLEUscore:.4f}")
print()
print(f"METEOR Score: {mean_meteor_score:.4f}")
print(f"BERTScore Precision: {mean_bert_precision:.4f}")
print(f"BERTScore Recall: {mean_bert_recall:.4f}")
print(f"BERTScore F1: {mean_bert_f1:.4f}")

In [ ]:
# Print input text, predicted summary, and actual summary for one example
example = dataset[40]
inputs = tokenizer(example["Article"], return_tensors="pt", max_length=1024, truncation=True)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(example["Headlines"], return_tensors="pt", max_length=15, truncation=True)

# Generate summary
outputs = model.generate(**inputs)
generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Input Text:", example["Article"])
print("Predicted Headline:", generated_summary)
print("Actual Headline:", example["Headlines"])
